# Лабораторная работа 5. Pandas.
  
Художественный фильм Титаник режиссера Джеймса Кэмерона славен тем, что идёт три часа и фильме кругом вода. В этой связи многие зрители покидают кинозал для посещения уборной.  
В некотором недалеком будущем 7D кинотеатр “ДК Академия” запускает показ ремастера фильма в формате 7D GALACTIC HD. И нанимает специалиста по обработке данных и машинному обучению, чтобы он рассчитал нагрузку на туалетные комнаты во время сеанса. Этот специалист вы! В первую очередь вам необходимо отфильтровать и должным образом подготовить данные, которые вам предоставил кинотеатр. За работу!  

<img src="images/LessonsI/Cinema7D.png" alt="Cinema7D" height=50% width=50%>

## Лабораторная 5.1
Данные, которые предоставил кинотеатр находятся в файлах *cinema_sessions.csv* и *titanic_with_labels*  
  
1. Пол (sex): отфильтровать строки, где пол не указан, преобразовать оставшиеся в число 0/1;  
2. Номер ряда в зале (row_number): заполнить вместо NAN максимальным значением ряда;  
3. Количество выпитого в литрах (liters_drunk): отфильтровать отрицательные значения и нереально большие значения (выбросы). Вместо них заполнить средним.  

## Лабораторная 5.2
4. Возраст (age): разделить на 3 группы: дети (до 18 лет), взрослые (18 - 50), пожилые (50+). закодировать в виде трёх столбцов с префиксом age_. Старый столбец с age удалить;
5. Напиток (drink): преобразовать в число 0/1 был ли этот напиток хмельным;
6. Номер чека (check_number): надо сопоставить со второй таблицей со временем сеанса. И закодировать в виде трёх столбцов, был ли это утренний (morining) сеанс, дневной (day) или вечерний (evening).


In [3]:
import pandas as pd
import numpy as np

# Загрузка данных
titanic_with_labels = pd.read_csv("data/titanic_with_labels.csv", sep=' ')
cinema_sessions = pd.read_csv("data/cinema_sessions.csv", sep=' ')

# 1. Обработка пола (sex)
def process_sex(df):
    # Приводим значения пола к одному стандарту: "м" -> "male", "ж" -> "female", "M" -> "male"
    df['sex'] = df['sex'].str.lower().map({
        'м': 1, 'm': 1, 'ж': 0, 'f': 0
    })
    
    # Удаляем строки, где пол не указан или не удалось сопоставить
    df = df[df['sex'].notna()]
    
    return df

titanic_with_labels = process_sex(titanic_with_labels)

# 2. Обработка номеров рядов (row_number)
def fill_max_row(df):
    # Находим максимальный номер ряда
    max_row = df['row_number'].max()
    
    # Заполняем пропущенные значения максимальным номером ряда
    df['row_number'] = df['row_number'].fillna(max_row)
    
    return df

titanic_with_labels = fill_max_row(titanic_with_labels)

# 3. Обработка количества выпитого в литрах (liters_drunk)
def filter_liters(df):
    # Удаляем отрицательные значения и выбросы (например, больше 10 литров)
    mean_liters = df['liters_drunk'][(df['liters_drunk'] >= 0) & (df['liters_drunk'] <= 10)].mean()
    
    # Заменяем выбросы и отрицательные значения средним значением
    df['liters_drunk'] = np.where((df['liters_drunk'] < 0) | (df['liters_drunk'] > 10), mean_liters, df['liters_drunk'])
    
    return df

titanic_with_labels = filter_liters(titanic_with_labels)

# Приводим имена столбцов к нижнему регистру для согласованности
titanic_with_labels.columns = titanic_with_labels.columns.str.lower()
cinema_sessions.columns = cinema_sessions.columns.str.lower()

# 4. Объединение данных по столбцу check_number
merged_data = pd.merge(titanic_with_labels, cinema_sessions, on='check_number', how='inner')
titanic_with_labels = filter_liters(titanic_with_labels)

# 4. Обработка возраста (age)
def process_age(df):
    # Создаем новые столбцы для группировки по возрасту
    df['age_child'] = (df['age'] < 18).astype(int)
    df['age_adult'] = ((df['age'] >= 18) & (df['age'] <= 50)).astype(int)
    df['age_senior'] = (df['age'] > 50).astype(int)
    
    # Удаляем старый столбец с возрастом
    df.drop(columns=['age'], inplace=True)
    
    return df

titanic_with_labels = process_age(titanic_with_labels)

# 5. Обработка напитков (drink)
def process_drink(df):
    df['is_alcoholic'] = df['drink'].str.lower().map({
        'cola': 0,
        'fanta': 0,
        'beerbeer': 1  # добавьте другие напитки по мере необходимости
    })
    return df

titanic_with_labels = process_drink(titanic_with_labels)

# 6. Обработка номера чека (check_number)
def process_check_number(df, sessions):
    # Преобразуем время сессий в категории
    def categorize_time(row):
        time = pd.to_datetime(row['session_start']).time()
        if time < pd.to_datetime("12:00:00").time():
            return 'morning'
        elif time < pd.to_datetime("18:00:00").time():
            return 'day'
        else:
            return 'evening'
    
    sessions['session_category'] = sessions.apply(categorize_time, axis=1)
    
    # Объединяем данные по check_number и создаем dummy-переменные
    merged_sessions = pd.get_dummies(sessions['session_category'], prefix='session', drop_first=True)
    df = df.merge(sessions[['check_number']], on='check_number', how='left').join(merged_sessions)
    
    return df

titanic_with_labels = process_check_number(titanic_with_labels, cinema_sessions)

# Приводим имена столбцов к нижнему регистру для согласованности
titanic_with_labels.columns = titanic_with_labels.columns.str.lower()
cinema_sessions.columns = cinema_sessions.columns.str.lower()

# 7. Объединение данных по столбцу check_number
merged_data = pd.merge(titanic_with_labels, cinema_sessions, on='check_number', how='inner')

# Сохраняем объединенные данные в CSV
merged_data.to_csv('merged_data.csv', index=False)

print("Данные обработаны и объединены. Результат сохранен в 'merged_data.csv'.")


Данные обработаны и объединены. Результат сохранен в 'merged_data.csv'.
